In [1]:
from gohr import *

In [2]:
def train_speck_distinguisher(num_epochs, nr, n, depth=1):
    #create the network
    net = make_resnet(depth=depth, reg_param=10**-5);
    net.compile(optimizer='adam',loss='mse',metrics=['acc']);
    #generate training and validation data
    X, Y = make_train_data(n, nr);
    X_eval, Y_eval = make_train_data(n//10, nr);
    #create learnrate schedule
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    #train and evaluate
    h = net.fit(X,Y,epochs=num_epochs,batch_size=bs,validation_data=(X_eval, Y_eval), callbacks=[lr]);

    np.save('data_train_full.npy', X);
    np.save('data_train_labels.npy', Y);
    np.save('data_train_real_samples.npy', X[np.where(Y==1)]);

    np.save('data_test_full.npy', X_eval);
    np.save('data_test_labels.npy', Y_eval);
    np.save('data_test_real_samples.npy', X_eval[np.where(Y_eval==1)]);

    print("Best validation accuracy: ", np.max(h.history['val_acc']));

    return(X, Y);

In [4]:
!pwd

/opt/cryptanalysis_servers_shared_folder/NBEATSCipherDistinguisher/nbeats_statistical_test/gohr


In [3]:
X, Y = train_speck_distinguisher(10, 6, 10**7)

2022-04-16 09:17:24.935646: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-16 09:17:29.077469: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2022-04-16 09:17:29.079485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38414 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capability: 8.0
2022-04-16 09:17:29.081349: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 

Epoch 1/10


2022-04-16 09:17:38.643606: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-04-16 09:17:40.054543: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-04-16 09:17:41.294080: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2000/2000 [==============================] - 18s 6ms/step - loss: 0.1794 - acc: 0.7312 - val_loss: 0.1626 - val_acc: 0.7651 - lr: 0.0020
Epoch 2/10
2000/2000 [==============================] - 11s 5ms/step - loss: 0.1565 - acc: 0.7748 - val_loss: 0.1554 - val_acc: 0.7769 - lr: 0.0018
Epoch 3/10
2000/2000 [==============================] - 11s 6ms/step - loss: 0.1542 - acc: 0.7785 - val_loss: 0.1543 - val_acc: 0.7785 - lr: 0.0016
Epoch 4/10
2000/2000 [==============================] - 11s 6ms/step - loss: 0.1532 - acc: 0.7798 - val_loss: 0.1537 - val_acc: 0.7797 - lr: 0.0014
Epoch 5/10
2000/2000 [==============================] - 10s 5ms/step - loss: 0.1525 - acc: 0.7806 - val_loss: 0.1527 - val_acc: 0.7803 - lr: 0.0012
Epoch 6/10
2000/2000 [==============================] - 11s 5ms/step - loss: 0.1519 - acc: 0.7814 - val_loss: 0.1522 - val_acc: 0.7808 - lr: 9.4444e-04
Epoch 7/10
2000/2000 [==============================] - 11s 5ms/step - loss: 0.1513 - acc: 0.7821 - val_loss: 0.1517 - 

# `generalized_gohr` 

In [1]:
from keras.layers import Dense, Conv1D, Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation
from keras.layers import Concatenate, MaxPooling2D


In [4]:
input_neurons = 64
num_filters = 32

import numpy as np
img_sqrt = int(np.sqrt(input_neurons))

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv1D, Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation
from keras.layers import Concatenate, MaxPooling2D

model = keras.Sequential()
model.add(Input(shape=(input_neurons)))
model.add(Reshape((4,16))) 
model.add(Permute((2,1)))
model.add(Conv1D(num_filters*2, kernel_size=1, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(num_filters, kernel_size=3, padding='same'))
model.summary()

# model = keras.Sequential()
# model.add(Input(shape=(input_neurons)))
# model.add(Reshape((img_sqrt,img_sqrt)))
# model.add(Permute((2,1)))
# #model.add(Conv1D(num_filters*2, kernel_size=1, padding='same'))
# #model.add(Conv1D(num_filters, kernel_size=3, padding='same'))
# model.summary()

# Call model on a test input
x = tf.convert_to_tensor(np.arange(input_neurons))
x = tf.reshape(x, (1, input_neurons)) # to make x a valid model input, the first dimension has to be the number of samples (1)
y = model(x)
y

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 4, 16)             0         
                                                                 
 permute_2 (Permute)         (None, 16, 4)             0         
                                                                 
 conv1d_2 (Conv1D)           (None, 16, 64)            320       
                                                                 
 activation_1 (Activation)   (None, 16, 64)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 16, 32)            6176      
                                                                 
Total params: 6,496
Trainable params: 6,496
Non-trainable params: 0
_________________________________________________________________


2022-04-20 16:05:31.366320: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-04-20 16:05:32.568112: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


<tf.Tensor: shape=(1, 16, 32), dtype=float32, numpy=
array([[[ -2.6095324 ,   3.604008  ,   7.9089074 ,   0.41484582,
          -2.2607586 ,   0.96337605,   3.2733598 ,   3.1761317 ,
          -7.1126733 ,  -0.6546638 ,  -5.285961  ,   4.1520452 ,
           6.718466  ,   1.1615256 ,  -4.787424  ,  -5.5447054 ,
           2.642722  ,  11.261836  ,   5.363558  ,   7.823221  ,
         -10.500961  ,   4.74484   ,   8.375633  ,   7.7465806 ,
         -11.2287655 ,  12.0367565 ,   7.1719213 ,   0.18827105,
          -3.2608054 ,  -2.3985646 ,   6.828616  ,   3.0558486 ],
        [  4.287055  ,  -0.31594914,   4.2864065 ,   2.746269  ,
          -4.1474657 ,   4.1373544 ,  12.101219  ,   1.2856698 ,
          -9.978552  ,  -5.5343833 ,  -8.619227  ,   2.4149196 ,
           4.7569103 ,   0.18530636,   2.087522  , -10.924616  ,
          -0.726223  ,   8.565805  ,   7.7244377 ,   8.420779  ,
         -12.647728  ,  10.831197  ,   1.9039855 ,   5.440014  ,
         -14.281349  ,  16.78512   ,

In [5]:
model.layers[-1].weights

[<tf.Variable 'conv1d_3/kernel:0' shape=(3, 64, 32) dtype=float32, numpy=
 array([[[ 0.14354047,  0.02458915, -0.06277097, ..., -0.10283241,
           0.12815365, -0.12849376],
         [-0.10094772, -0.02960481, -0.01359944, ..., -0.06570765,
          -0.09763581,  0.02405043],
         [ 0.11872768, -0.08624876,  0.09596403, ..., -0.05429596,
          -0.10883072,  0.09438436],
         ...,
         [ 0.02743533,  0.11702055, -0.13368183, ...,  0.00112206,
          -0.08765683,  0.12213826],
         [ 0.14402303,  0.02894667, -0.12337376, ...,  0.12333024,
           0.09687178, -0.10157792],
         [-0.14417046,  0.12276313, -0.03695438, ...,  0.03988273,
           0.11471289,  0.03740819]],
 
        [[ 0.11059678, -0.122151  ,  0.09708235, ...,  0.07853787,
          -0.10125114, -0.02636334],
         [-0.11923363, -0.11846702,  0.0266353 , ..., -0.04778473,
           0.12043029, -0.10257222],
         [ 0.13244271,  0.08256088, -0.06939463, ...,  0.01165184,
          

In [10]:
input_neurons = 64
x = tf.convert_to_tensor(np.arange(input_neurons))
x = tf.reshape(x, (8,8))
x

<tf.Tensor: shape=(8, 8), dtype=int64, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])>

In [7]:
import tensorflow as tf
import numpy as np

In [1]:
def create_gohr_generalized_model(input_neurons=32, output_neurons=10, model_strength=1, set_memory_growth=True):
    """
    Epoch 1/10
    2022-04-20 11:18:40.482091: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
    2022-04-20 11:18:41.662304: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

    You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
    2022-04-20 11:18:42.889289: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
    2000/2000 [==============================] - 32s 13ms/step - loss: 0.1851 - acc: 0.7246 - val_loss: 0.1669 - val_acc: 0.7571
    Epoch 2/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1605 - acc: 0.7685 - val_loss: 0.1575 - val_acc: 0.7748
    Epoch 3/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1557 - acc: 0.7765 - val_loss: 0.1555 - val_acc: 0.7767
    Epoch 4/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1545 - acc: 0.7779 - val_loss: 0.1545 - val_acc: 0.7784
    Epoch 5/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1535 - acc: 0.7797 - val_loss: 0.1537 - val_acc: 0.7798
    Epoch 6/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1529 - acc: 0.7808 - val_loss: 0.1531 - val_acc: 0.7807
    Epoch 7/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1525 - acc: 0.7813 - val_loss: 0.1527 - val_acc: 0.7813
    Epoch 8/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1522 - acc: 0.7818 - val_loss: 0.1526 - val_acc: 0.7813
    Epoch 9/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1519 - acc: 0.7823 - val_loss: 0.1522 - val_acc: 0.7821
    Epoch 10/10
    2000/2000 [==============================] - 25s 12ms/step - loss: 0.1517 - acc: 0.7827 - val_loss: 0.1519 - val_acc: 0.7825
    Best validation accuracy:  0.7825149893760681
    """
    #--- prepare GPU
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices("GPU")
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    if set_memory_growth:
        tf.config.experimental.set_memory_growth(gpus[0], True)

    from keras.models import Model
    from keras.layers import Dense, Conv1D, Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation
    from keras.layers import Concatenate, MaxPooling2D
    from keras.regularizers import l2
    import numpy as np
    
    img_sqrt = int(np.sqrt(input_neurons))
    
    #num_blocks=2 
    num_filters=32*4
    d1=64*model_strength
    d2=64*model_strength
    #word_size=16
    ks=3
    depth=model_strength
    reg_param=10**-5       
    final_activation='sigmoid'
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.002, amsgrad=True) #'Adam'
    loss = 'mse' #'Huber'
    
    #loss = 'mse'

    # put input in square shape instead of word-like structure
    inp = Input(shape=(input_neurons,));
    #rs = Reshape((2 * num_blocks, word_size))(inp);
    rs = Reshape((img_sqrt, img_sqrt))(inp) # changed rs = Reshape((img_sqrt, img_sqrt, 1))(inp)
    
    #---- search correlations in one direction
    conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(rs); # changed
    conv0 = BatchNormalization()(conv0);
    conv0 = Activation('relu')(conv0);
    #add residual blocks 
    shortcut = conv0;
    for i in range(depth):
        conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
        conv1 = BatchNormalization()(conv1);
        conv1 = Activation('relu')(conv1);
        conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
        conv2 = BatchNormalization()(conv2);
        conv2 = Activation('relu')(conv2);
        shortcut = Add()([shortcut, conv2]);
        
    #---- search correlations in the other direction
    perm = Permute((2,1))(rs); 
    conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm); # changed
    conv0 = BatchNormalization()(conv0);
    conv0 = Activation('relu')(conv0);
    shortcut1 = conv0;
    for i in range(depth):
        conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut1);
        conv1 = BatchNormalization()(conv1);
        conv1 = Activation('relu')(conv1);
        conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
        conv2 = BatchNormalization()(conv2);
        conv2 = Activation('relu')(conv2);
        shortcut1 = Add()([shortcut1, conv2]);

    #---- bring them together
    shortcut = Concatenate()([shortcut, shortcut1])#Add()([shortcut, shortcut1])
    #add prediction head
    flat1 = Flatten()(shortcut);
    dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
    dense1 = BatchNormalization()(dense1);
    dense1 = Activation('relu')(dense1);
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
    dense2 = BatchNormalization()(dense2);
    dense2 = Activation('relu')(dense2);
    out = Dense(output_neurons, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);
    model = Model(inputs=inp, outputs=out);
    #-------------#-------------#-------------#-------------#
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  run_eagerly=False, metrics=['acc'])  
        
#     from gohr import cyclic_lr
#     from keras.callbacks import LearningRateScheduler
#     lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    
    model.callbacks = []# [lr]
    return model

In [4]:
def create_gohr_generalized_model(input_neurons=32, output_neurons=10, model_strength=1, set_memory_growth=True):
    """
    CHANGE ME
    """
    #--- prepare GPU
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices("GPU")
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    if set_memory_growth:
        tf.config.experimental.set_memory_growth(gpus[0], True)

    from keras.models import Model
    from keras.layers import Dense, Conv1D, Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation
    from keras.layers import Concatenate, MaxPooling2D
    from keras.regularizers import l2
    import numpy as np
    
    img_sqrt = int(np.sqrt(input_neurons))
    
    #num_blocks=2 
    num_filters=32*4
    d1=64*model_strength
    d2=64*model_strength
    #word_size=16
    ks=3
    depth=model_strength
    reg_param=10**-5       
    final_activation='sigmoid'
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.002, amsgrad=True) #'Adam'
    loss = 'mse' #'Huber'
    
    #loss = 'mse'

    # put input in square shape instead of word-like structure
    inp = Input(shape=(input_neurons,));
    #rs = Reshape((2 * num_blocks, word_size))(inp);
    rs = Reshape((img_sqrt, img_sqrt))(inp) # changed rs = Reshape((img_sqrt, img_sqrt, 1))(inp)
    rs = Permute((2,1))(rs)
    #---- search correlations in one direction
    conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(rs); # changed
    conv0 = BatchNormalization()(conv0);
    conv0 = Activation('relu')(conv0);
    #add residual blocks 
    shortcut = conv0;
    for i in range(depth):
        conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
        conv1 = BatchNormalization()(conv1);
        conv1 = Activation('relu')(conv1);
        conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
        conv2 = BatchNormalization()(conv2);
        conv2 = Activation('relu')(conv2);
        shortcut = Add()([shortcut, conv2]);
        
    #---- search correlations in the other direction
#     perm = Permute((2,1))(rs); 
#     conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm); # changed
#     conv0 = BatchNormalization()(conv0);
#     conv0 = Activation('relu')(conv0);
#     shortcut1 = conv0;
#     for i in range(depth):
#         conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut1);
#         conv1 = BatchNormalization()(conv1);
#         conv1 = Activation('relu')(conv1);
#         conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
#         conv2 = BatchNormalization()(conv2);
#         conv2 = Activation('relu')(conv2);
#         shortcut1 = Add()([shortcut1, conv2]);

    #---- bring them together
    #shortcut = Concatenate()([shortcut, shortcut1])#Add()([shortcut, shortcut1])
    #add prediction head
    flat1 = Flatten()(shortcut);
    dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
    dense1 = BatchNormalization()(dense1);
    dense1 = Activation('relu')(dense1);
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
    dense2 = BatchNormalization()(dense2);
    dense2 = Activation('relu')(dense2);
    out = Dense(output_neurons, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);
    model = Model(inputs=inp, outputs=out);
    #-------------#-------------#-------------#-------------#
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  run_eagerly=False, metrics=['acc'])  
        
#     from gohr import cyclic_lr
#     from keras.callbacks import LearningRateScheduler
#     lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    
    model.callbacks = []# [lr]
    return model

In [5]:
def train_speck_distinguisher(num_epochs, nr, n, depth=1):
    
    from gohr import bs

    #create the network
    net = create_gohr_generalized_model(input_neurons=64, output_neurons=1, model_strength=1);
    
    import numpy as np
    #net.compile(optimizer='adam',loss='mse',metrics=['acc']);
    #generate training and validation data
    X = np.load('data_train_full.npy')
    Y = np.load('data_train_labels.npy')
    X_eval = np.load('data_test_full.npy')
    Y_eval = np.load('data_test_labels.npy')
    #create learnrate schedule
    #train and evaluate
    h = net.fit(X,Y,epochs=num_epochs,batch_size=bs,validation_data=(X_eval, Y_eval));

    print("Best validation accuracy: ", np.max(h.history['val_acc']));

    return(X, Y);

In [6]:
# with kernel regularizer  77.4% val_acc
#    (without kernel regularizer: 77.0% val_acc)
train_speck_distinguisher(10, 6, 10**7)

Epoch 1/10
2000/2000 [==============================] - 16s 7ms/step - loss: 0.1867 - acc: 0.7205 - val_loss: 0.1672 - val_acc: 0.7569
Epoch 2/10
2000/2000 [==============================] - 13s 7ms/step - loss: 0.1627 - acc: 0.7634 - val_loss: 0.1612 - val_acc: 0.7664
Epoch 3/10
2000/2000 [==============================] - 14s 7ms/step - loss: 0.1583 - acc: 0.7712 - val_loss: 0.1567 - val_acc: 0.7746
Epoch 4/10
2000/2000 [==============================] - 14s 7ms/step - loss: 0.1552 - acc: 0.7768 - val_loss: 0.1550 - val_acc: 0.7772
Epoch 5/10
1524/2000 [=====================>........] - ETA: 2s - loss: 0.1543 - acc: 0.7778

KeyboardInterrupt: 